# TTT Model


In [ ]:
# Mount user's Google Drive
#from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Set desired working folder
# %cd '/content/drive/MyDrive/BAW Colab Notebooks/IMQCAM/Ti6Al4V_rev_0'

# Set the name of the csv file for time/temperature history at each position
history = 'sample_z_time_temp.csv'

In [ ]:
import math as m
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
from scipy.signal import find_peaks
from scipy.signal import peak_widths
import os
import statistics as s
from datetime import datetime
import pytz

import NGFunctions as NG
import SinglePointSTK as SP


In [ ]:
# Specify the position time/temperature history
df = pd.read_csv(history, header=0,
              names=["z position", "Time (s)", "Temperature"])

df = df.pivot(index='Time (s)', columns='z position', values='Temperature')

# Read the data into lists
Times = list(df.index.values)
zpos = list(df.columns.values)

# Convert z from meters to millimeters
zpos = [10**3*x for x in zpos]

# Assuming a regular grid
dz = zpos[1] - zpos[0]

In [ ]:
# This cell computes the fractions of microstructure constituents for each position

Fraca = []
FracGB = []
FracBW = []
FracCOL = []
FracMASS = []
FracMART = []
Tlath = []

for c in df.columns:
    TempC = df[c]
    # Run the single point function
    Thermal = SP.ThermalStep(Times, TempC)
    Fraca.append(Thermal[0])
    FracGB.append(Thermal[1])
    FracBW.append(Thermal[2])
    FracCOL.append(Thermal[3])
    FracMASS.append(Thermal[4])
    FracMART.append(Thermal[5])
    Tlath.append(Thermal[6])

In [ ]:
# This cell outputs results

# This output is at the end of the time-temperature history
output = {'zpos[mm]':zpos, 'fracalpha':Fraca, 'fracGB':FracGB, 'fracBW':FracBW, 'fracCOL':FracCOL, 'fracMASS':FracMASS,  'fracMart':FracMART, 'tlath':Tlath}
dfout = pd.DataFrame(output)


# Output to a csv file using the time/temp history file and current date/time as a unique identifier
tz_NY = pytz.timezone('America/New_York')                                                   # Get the timezone object for New York (Eastern time zone)
now = datetime.now(tz_NY)                                                                   # Get the current time in New York
out_file = history.removesuffix('.csv') +'_out_' + now.strftime("%d%m%y_%H%M%S") + '.csv'   # Set the outputfilenama
dfout.to_csv(out_file)                                                                      # Write *.csv file